In [1]:
import numpy as np
import keras
from keras.models import Model, load_model
from keras.layers import Dense, Input, Concatenate, Dropout, Add, Lambda
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import os
from keras.engine.topology import Layer
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
os.environ["CUDA_VISIBLE_DEVICES"]="7"



Using TensorFlow backend.


In [ ]:
DATA = "ARTINT"

In [3]:
#load train data
train_img_filename =  pickle.load(open(DATA+"_train_img_filename_for_img_modal_vector.p", "rb"))
train_img_feature_vector = np.load(DATA+'_train_img_modal_vector.npy')
train_caption_feature_vector = pickle.load(open(DATA+"_train_image_caption_feature_vector.p", "rb"))
train_text_filename = pickle.load(open(DATA+"_train_abstract_filename.p", "rb"))
train_text_feature_vector =  np.load(DATA+'_train_abstract_modality_feature.npy')


In [4]:
#load test data
test_img_filename = pickle.load(open(DATA+"_test_img_filename_for_img_modal_vector.p", "rb"))
test_img_feature_vector = np.load(DATA+'_test_img_modal_vector.npy')
test_caption_feature_vector = pickle.load(open(DATA+"_test_image_caption_feature_vector.p", "rb"))
test_text_filename = pickle.load(open(DATA+"_test_abstract_filename.p", "rb"))
test_text_feature_vector = np.load(DATA+'_test_abstract_modality_feature.npy')


In [5]:
fv_img = []
fv_img_caption = []
fv_full_text = []
train_labels = []
for i in range(len(train_img_filename)):
    for j in range(len(train_text_filename)):
        if train_img_filename[i] in train_text_filename[j]:
            if train_text_filename[j] not in train_caption_feature_vector.keys():
                continue
            fv_img.append(train_img_feature_vector[i])
            fv_img_caption.append(train_caption_feature_vector[train_text_filename[j]])
            fv_full_text.append(train_text_feature_vector[j])
            label = 1.0
            if 'neg_' in train_img_filename[i]:
                label = 0.0
            train_labels.append(label)

fv_img = np.array(fv_img)
fv_img_caption = np.array(fv_img_caption)
fv_full_text = np.array(fv_full_text)


In [6]:
fv_img_test = []
fv_img_test_caption = []
fv_full_text_test = []
test_labels = []
for i in range(len(test_img_filename)):
    for j in range(len(test_text_filename)):
        if test_img_filename[i] in test_text_filename[j]:
            if test_text_filename[j] not in test_caption_feature_vector.keys():
                continue
            fv_img_test.append(test_img_feature_vector[i])
            fv_img_test_caption.append(test_caption_feature_vector[test_text_filename[j]])
            fv_full_text_test.append(test_text_feature_vector[j])
            label = 1.0
            if 'neg_' in test_img_filename[i]:
                label = 0.0
            test_labels.append(label)
            
fv_img_test = np.array(fv_img_test)
fv_img_test_caption = np.array(fv_img_test_caption)
fv_full_text_test = np.array(fv_full_text_test)


In [7]:
print(fv_img.shape)
print(fv_img_caption.shape)
print(fv_full_text.shape)
print(fv_img_test.shape)
print(fv_img_test_caption.shape)
print(fv_full_text_test.shape)

(3056, 4096)
(3056, 5716)
(3056, 100)
(768, 4096)
(768, 5716)
(768, 100)


In [8]:
#model

IMG_FEATURES = Input(shape = (4096,))
IMG_CAPTIONS = Input(shape = (5716,))
FULL_TEXT = Input(shape = (100,))

B = Concatenate(axis = 1)([IMG_FEATURES, IMG_CAPTIONS, FULL_TEXT])

P = Dense(512, activation = 'relu')(B)

alpha = Dense(3, activation = 'softmax')(P)

F = Lambda(lambda x : alpha[:,0:1]*IMG_FEATURES)(alpha)
G = Lambda(lambda x : alpha[:,1:2]*IMG_CAPTIONS)(alpha)
H = Lambda(lambda x : alpha[:,2:3]*FULL_TEXT)(alpha)
Y = Concatenate(axis = -1)([F,G,H])

Y = Dense(2000, activation = 'relu')(Y)
Y = Dropout(rate = 0.2)(Y)

Y = Dense(512, activation = 'relu')(Y)		#, kernel_regularizer = regularizers.l2(0.01)
Y = Dropout(rate = 0.25)(Y)

Y = Dense(512, activation = 'relu')(Y)
Y = Dropout(rate = 0.2)(Y)

Y = Dense(1, activation = 'sigmoid')(Y)

model = Model(inputs = [IMG_FEATURES, IMG_CAPTIONS, FULL_TEXT], outputs = Y)

In [9]:
checkpoint = ModelCheckpoint(DATA+'model-img-caption-abstract-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

results = model.fit( [fv_img,fv_img_caption,fv_full_text],train_labels, epochs= 20, batch_size = 64,validation_data=([fv_img_test,fv_img_test_caption,fv_full_text_test],test_labels),callbacks=[checkpoint])


Train on 3056 samples, validate on 768 samples
Epoch 1/20
3056/3056 [==============================] - 4s 1ms/step - loss: 0.1934 - acc: 0.9310 - val_loss: 0.1959 - val_acc: 0.9232

Epoch 00001: val_loss improved from inf to 0.19585, saving model to model-img-caption-abstract-best.h5
Epoch 2/20
3056/3056 [==============================] - 1s 320us/step - loss: 0.1442 - acc: 0.9454 - val_loss: 0.1889 - val_acc: 0.9180

Epoch 00002: val_loss improved from 0.19585 to 0.18892, saving model to model-img-caption-abstract-best.h5
Epoch 3/20
3056/3056 [==============================] - 1s 314us/step - loss: 0.1357 - acc: 0.9486 - val_loss: 0.1957 - val_acc: 0.9167

Epoch 00003: val_loss did not improve from 0.18892
Epoch 4/20
3056/3056 [==============================] - 1s 317us/step - loss: 0.1306 - acc: 0.9568 - val_loss: 0.2016 - val_acc: 0.9167

Epoch 00004: val_loss did not improve from 0.18892
Epoch 5/20
3056/3056 [==============================] - 1s 318us/step - loss: 0.1238 - acc: 0.9

In [10]:
predicted = model.predict([fv_img_test,fv_img_test_caption,fv_full_text_test])

In [11]:
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [12]:
print("fscore",f1_score(test_labels, predicted, average="macro"))
print("precision",precision_score(test_labels, predicted, average="macro"))
print("recall",recall_score(test_labels, predicted, average="macro"))    

x = classification_report(test_labels,predicted)
print(x)

fscore 0.9059412677244645
precision 0.9062091503267974
recall 0.9057229898814058
             precision    recall  f1-score   support

        0.0       0.91      0.90      0.90       364
        1.0       0.91      0.92      0.91       404

avg / total       0.91      0.91      0.91       768



In [13]:
model.save(DATA+'image-caption-abstract-model')